### Imports and utility

In [45]:
# Standard library imports
import os
import sys
import importlib
import numpy as np
import random
import cv2
from tqdm import tqdm

# Add project-specific paths
sys.path.append(os.path.join(os.getcwd(), 'common'))

# Import custom modules
import utility
import embedding
import detection
import grt

# Reload module for updates
importlib.reload(utility)
importlib.reload(embedding)
importlib.reload(detection)
importlib.reload(grt)

<module 'grt' from 'c:\\Users\\Utente\\Documents\\Università\\Master\\MultimediaDataSecurity\\CTM\\multimedia-data-security\\common\\grt.py'>

In [32]:
def robustness(wpsnr):
    if wpsnr >= 53:
        points = 0
    elif wpsnr >= 50:
        points = 1
    elif wpsnr >= 47:
        points = 2
    elif wpsnr >= 44:
        points = 3
    elif wpsnr >= 41:
        points = 4
    elif wpsnr >= 38:
        points = 5
    else:
        points = 6
    return points

def imperceptibility(wpsnr):
    if wpsnr >= 66:
        points = 6
    elif wpsnr >= 62:
        points = 5
    elif wpsnr >= 58:
        points = 4
    elif wpsnr >= 54:
        points = 3
    elif wpsnr >= 50:
        points = 2
    elif wpsnr >= 35:
        points = 1
    else:
        points = 0
    return points

### Embedding

In [18]:
N = 20
images = []
watermarked = []
alpha = 0.48
max_layer = 1
avg = 0
min = 9999999999

# Randomly select N images
img_folder = 'sample_imgs'
img_files =  [f for f in os.listdir(img_folder) if f.endswith(('.bmp'))]
img_files = random.sample(img_files, N)
for file in img_files:
    img_path = os.path.join(img_folder, file)
    images.append(cv2.imread(img_path, 0))
images = np.array(images)

mark = np.load('ammhackati.npy')
   
progress_bar = tqdm(images, desc = "Embedding the watermark in " + str(N) + " images")
for index, img in enumerate(progress_bar):
    watermarked.append(embedding.embedding(img, mark, alpha, max_layer))
    wpsnr = utility.wpsnr(img, watermarked[index])
    avg += wpsnr
    if wpsnr < min:
        min = wpsnr
        
avg = avg / N
print("\nAverage wpsnr: " + str(avg) + ' --> ' + str(imperceptibility(avg)) + ' imperceptibility points')
print("Min wpsnr: " + str(min) + ' --> ' + str(imperceptibility(min)) + ' imperceptibility points')

Embedding the watermark in 2 images: 100%|██████████| 2/2 [00:02<00:00,  1.23s/it]


Average wpsnr: 62.685427702167466 --> 5 imperceptibility points
Min wpsnr: 61.56243603786693 --> 4 imperceptibility points


### Global attacks

1. Blur using gaussian filters

In [25]:
progress_bar = tqdm(watermarked, desc = "Blurring " + str(N) + " images")
history = []
successful_attacks = []
total = 0
succ = 0
avg = 0
max = 0

for index, wm in enumerate(progress_bar):
    for sigma in np.arange(0, 5, 0.1, dtype = np.float16):
        attacked, attack_name, usd = grt.blur_gauss(wm, sigma)
        detected = detection.detection(images[index], wm, attacked, alpha, max_layer)
        wpsnr_attacked = utility.wpsnr(wm, attacked)
        log = {"image" : img_files[index],
                "attack" : attack_name,
                "param" : usd,
                "wpsnr" : wpsnr_attacked,
                "detection" : detected}
        history.append(log)
        if detected == 0 and wpsnr >= 35:
            total += wpsnr_attacked
            succ +=1
            successful_attacks.append(log)
            if wpsnr_attacked > max:
                max = wpsnr_attacked
            break

if succ > 0:
    avg = total/succ
    print("\nAverage wpsnr of successful attacks: " + str(avg) + ' --> ' + str(robustness(avg)) + ' robustness points')
    print("Max wpsnr of successful attacks: " + str(max) + ' --> ' + str(robustness(max)) + ' robustness points\n') 
else:
    print("\nThis attack is not very effective...\n")
 
for s in successful_attacks:
    print(s) 

Blurring 2 images: 100%|██████████| 2/2 [00:18<00:00,  9.17s/it]


Average wpsnr of successful attacks: 52.497455145670365
Max wpsnr of successful attacks: 54.47320746355105 --> 0 robustness points

{'image': '0087.bmp', 'attack': 'blur_gauss', 'param': {'sigma': 0.5}, 'wpsnr': 54.47320746355105, 'detection': 0}
{'image': '0029.bmp', 'attack': 'blur_gauss', 'param': {'sigma': 0.5}, 'wpsnr': 50.52170282778968, 'detection': 0}


2. Blur using median filters

In [26]:
progress_bar = tqdm(watermarked, desc = "Blurring " + str(N) + " images")
history = []
successful_attacks = []
total = 0
succ = 0
avg = 0
max = 0

for index, wm in enumerate(progress_bar):
    for height in np.arange(1, 15, 2):
        for width in np.arange(1, 15, 2):
            attacked, attack_name, usd = grt.blur_median(wm, [height, width])
            detected = detection.detection(images[index], wm, attacked, alpha, max_layer)
            wpsnr_attacked = utility.wpsnr(wm, attacked)
            log = {"image" : img_files[index],
                    "attack" : attack_name,
                    "param" : usd,
                    "wpsnr" : wpsnr_attacked,
                    "detection" : detected}
            history.append(log)
            if detected == 0 and wpsnr >= 35:
                total += wpsnr_attacked
                succ +=1
                successful_attacks.append(log)
                if wpsnr_attacked > max:
                    max = wpsnr_attacked
                break

if succ > 0:
    avg = total/succ
    print("\nAverage wpsnr of successful attacks: " + str(avg) + ' --> ' + str(robustness(avg)) + ' robustness points')
    print("Max wpsnr of successful attacks: " + str(max) + ' --> ' + str(robustness(max)) + ' robustness points\n') 
else:
    print("\nThis attack is not very effective...\n")

for s in successful_attacks:
    print(s) 

Blurring 2 images: 100%|██████████| 2/2 [04:45<00:00, 142.55s/it]


Average wpsnr of successful attacks: 40.817217670811345
Max wpsnr of successful attacks: 46.08652812892195 --> 3 robustness points

{'image': '0087.bmp', 'attack': 'blur_median', 'param': {'size': [1, 5]}, 'wpsnr': 43.426989413837305, 'detection': 0}
{'image': '0087.bmp', 'attack': 'blur_median', 'param': {'size': [3, 3]}, 'wpsnr': 46.08652812892195, 'detection': 0}
{'image': '0087.bmp', 'attack': 'blur_median', 'param': {'size': [5, 1]}, 'wpsnr': 40.31324631684633, 'detection': 0}
{'image': '0087.bmp', 'attack': 'blur_median', 'param': {'size': [7, 1]}, 'wpsnr': 35.64198958838507, 'detection': 0}
{'image': '0029.bmp', 'attack': 'blur_median', 'param': {'size': [3, 3]}, 'wpsnr': 42.17102272513812, 'detection': 0}
{'image': '0029.bmp', 'attack': 'blur_median', 'param': {'size': [5, 3]}, 'wpsnr': 37.2635298517393, 'detection': 0}


3. Adding white gaussian noise (AWGN)

In [33]:
progress_bar = tqdm(watermarked, desc = "Applying AWGN to " + str(N) + " images")
history = []
successful_attacks = []
total = 0
succ = 0
avg = 0
max = 0

for index, wm in enumerate(progress_bar):
    for std in range(0, 110, 10):
        attacked, attack_name, usd = grt.awgn(wm, 0, std, 100)
        detected = detection.detection(images[index], wm, attacked, alpha, max_layer)
        wpsnr_attacked = utility.wpsnr(wm, attacked)
        log = {"image" : img_files[index],
                "attack" : attack_name,
                "param" : usd,
                "wpsnr" : wpsnr_attacked,
                "detection" : detected}
        history.append(log)
        if detected == 0 and wpsnr >= 35:
            total += wpsnr_attacked
            succ +=1
            successful_attacks.append(log)
            if wpsnr_attacked > max:
                max = wpsnr_attacked
            break

if succ > 0:
    avg = total/succ
    print("\nAverage wpsnr of successful attacks: " + str(avg) + ' --> ' + str(robustness(avg)) + ' robustness points')
    print("Max wpsnr of successful attacks: " + str(max) + ' --> ' + str(robustness(max)) + ' robustness points\n') 
else:
    print("\nThis attack is not very effective...\n")
 
for s in successful_attacks:
    print(s)

Applying AWGN to 2 images: 100%|██████████| 2/2 [00:31<00:00, 15.82s/it]


Average wpsnr of successful attacks: 37.35400041119342 --> 6 robustness points
Max wpsnr of successful attacks: 37.35400041119342 --> 6 robustness points

{'image': '0087.bmp', 'attack': 'awgn', 'param': {'mean': 0, 'std': 20, 'seed': 100}, 'wpsnr': 37.35400041119342, 'detection': 0}


4. JPEG compression

In [34]:
progress_bar = tqdm(watermarked, desc = "Compressing " + str(N) + " images")
history = []
successful_attacks = []
total = 0
succ = 0
avg = 0
max = 0

for index, wm in enumerate(progress_bar):
    for qf in range(0, 100, 2):
        attacked, attack_name, usd = grt.jpeg_compression(wm, (100 - qf))
        detected = detection.detection(images[index], wm, attacked, alpha, max_layer)
        wpsnr_attacked = utility.wpsnr(wm, attacked)
        log = {"image" : img_files[index],
                "attack" : attack_name,
                "param" : usd,
                "wpsnr" : wpsnr_attacked,
                "detection" : detected}
        history.append(log)
        if detected == 0 and wpsnr >= 35:
            total += wpsnr_attacked
            succ +=1
            successful_attacks.append(log)
            if wpsnr_attacked > max:
                max = wpsnr_attacked
            break

if succ > 0:
    avg = total/succ
    print("\nAverage wpsnr of successful attacks: " + str(avg) + ' --> ' + str(robustness(avg)) + ' robustness points')
    print("Max wpsnr of successful attacks: " + str(max) + ' --> ' + str(robustness(max)) + ' robustness points\n') 
else:
    print("\nThis attack is not very effective...\n")

for s in successful_attacks:
    print(s)

Compressing 2 images: 100%|██████████| 2/2 [01:33<00:00, 46.72s/it]


Average wpsnr of successful attacks: 51.446251014756726 --> 1 robustness points
Max wpsnr of successful attacks: 52.143191844857746 --> 1 robustness points

{'image': '0087.bmp', 'attack': 'jpeg_compression', 'param': {'qf': 55}, 'wpsnr': 52.143191844857746, 'detection': 0}
{'image': '0029.bmp', 'attack': 'jpeg_compression', 'param': {'qf': 45}, 'wpsnr': 50.749310184655705, 'detection': 0}


5. Resizing

In [36]:
progress_bar = tqdm(watermarked, desc = "Resizing " + str(N) + " images")
history = []
successful_attacks = []
total = 0
succ = 0
avg = 0
max = 0

for index, wm in enumerate(progress_bar):
    for scale in np.arange(0, 1, 0.01, dtype = np.float16):
        attacked, attack_name, usd = grt.resize(wm, (1 - scale))
        detected = detection.detection(images[index], wm, attacked, alpha, max_layer)
        wpsnr_attacked = utility.wpsnr(wm, attacked)
        log = {"image" : img_files[index],
                "attack" : attack_name,
                "param" : usd,
                "wpsnr" : wpsnr_attacked,
                "detection" : detected}
        history.append(log)
        if detected == 0 and wpsnr >= 35:
            total += wpsnr_attacked
            succ +=1
            successful_attacks.append(log)
            if wpsnr_attacked > max:
                max = wpsnr_attacked
            break

if succ > 0:
    avg = total/succ
    print("\nAverage wpsnr of successful attacks: " + str(avg) + ' --> ' + str(robustness(avg)) + ' robustness points')
    print("Max wpsnr of successful attacks: " + str(max) + ' --> ' + str(robustness(max)) + ' robustness points\n') 
else:
    print("\nThis attack is not very effective...\n")
 
for s in successful_attacks:
    print(s) 

Resizing 2 images: 100%|██████████| 2/2 [10:09<00:00, 304.86s/it]


This attack is not very effective...



### Localized attacks

1. Blur edges using gaussian filters 

In [38]:
progress_bar = tqdm(watermarked, desc = "Blurring edges of " + str(N) + " images")
history = []
successful_attacks = []
total = 0
succ = 0
avg = 0
max = 0

for index, wm in enumerate(progress_bar):
    for edge_func in range(0, 2):
        for sigma in np.arange(0, 5, 0.1, dtype = np.float16):        
            attacked, attack_name, usd = grt.gauss_edge(wm, sigma, edge_func)
            detected = detection.detection(images[index], wm, attacked, alpha, max_layer)
            wpsnr_attacked = utility.wpsnr(wm, attacked)
            log = {"image" : img_files[index],
                    "attack" : attack_name,
                    "param" : usd,
                    "wpsnr" : wpsnr_attacked,
                    "detection" : detected}
            history.append(log)
            if detected == 0 and wpsnr >= 35:
                total += wpsnr_attacked
                succ +=1
                successful_attacks.append(log)
                if wpsnr_attacked > max:
                    max = wpsnr_attacked
                break

if succ > 0:
    avg = total/succ
    print("\nAverage wpsnr of successful attacks: " + str(avg) + ' --> ' + str(robustness(avg)) + ' robustness points')
    print("Max wpsnr of successful attacks: " + str(max) + ' --> ' + str(robustness(max)) + ' robustness points\n') 
else:
    print("\nThis attack is not very effective...\n")
 
for s in successful_attacks:
    print(s) 

Blurring edges of 2 images: 100%|██████████| 2/2 [01:05<00:00, 32.74s/it]


Average wpsnr of successful attacks: 51.629031994203956 --> 1 robustness points
Max wpsnr of successful attacks: 56.625201986855174 --> 0 robustness points

{'image': '0087.bmp', 'attack': 'gauss_edge', 'param': {'sigma': 0.5, 'edge_func': 0}, 'wpsnr': 56.625201986855174, 'detection': 0}
{'image': '0087.bmp', 'attack': 'gauss_edge', 'param': {'sigma': 0.8, 'edge_func': 1}, 'wpsnr': 52.05988177497166, 'detection': 0}
{'image': '0029.bmp', 'attack': 'gauss_edge', 'param': {'sigma': 0.5, 'edge_func': 0}, 'wpsnr': 51.729198334967485, 'detection': 0}
{'image': '0029.bmp', 'attack': 'gauss_edge', 'param': {'sigma': 0.9, 'edge_func': 1}, 'wpsnr': 46.10184588002149, 'detection': 0}


2. Blur edges using median filters

In [39]:
progress_bar = tqdm(watermarked, desc = "Blurring edges of " + str(N) + " images")
history = []
successful_attacks = []
total = 0
succ = 0
avg = 0
max = 0

for index, wm in enumerate(progress_bar):
    for edge_func in range(0, 2):
        for height in np.arange(1, 15, 2):
            for width in np.arange(1, 15, 2):       
                attacked, attack_name, usd = grt.median_edge(wm, [height, width], edge_func)
                detected = detection.detection(images[index], wm, attacked, alpha, max_layer)
                wpsnr_attacked = utility.wpsnr(wm, attacked)
                log = {"image" : img_files[index],
                        "attack" : attack_name,
                        "param" : usd,
                        "wpsnr" : wpsnr_attacked,
                        "detection" : detected}
                history.append(log)
                if detected == 0 and wpsnr >= 35:
                    total += wpsnr_attacked
                    succ +=1
                    successful_attacks.append(log)
                    if wpsnr_attacked > max:
                        max = wpsnr_attacked
                    break

if succ > 0:
    avg = total/succ
    print("\nAverage wpsnr of successful attacks: " + str(avg) + ' --> ' + str(robustness(avg)) + ' robustness points')
    print("Max wpsnr of successful attacks: " + str(max) + ' --> ' + str(robustness(max)) + ' robustness points\n') 
else:
    print("\nThis attack is not very effective...\n")
 
for s in successful_attacks:
    print(s) 

Blurring edges of 2 images: 100%|██████████| 2/2 [06:57<00:00, 208.51s/it]


Average wpsnr of successful attacks: 40.4230855592633 --> 5 robustness points
Max wpsnr of successful attacks: 48.18372154663639 --> 2 robustness points

{'image': '0087.bmp', 'attack': 'median_edge', 'param': {'size': [1, 7], 'edge_func': 0}, 'wpsnr': 41.672705759368405, 'detection': 0}
{'image': '0087.bmp', 'attack': 'median_edge', 'param': {'size': [3, 3], 'edge_func': 0}, 'wpsnr': 48.18372154663639, 'detection': 0}
{'image': '0087.bmp', 'attack': 'median_edge', 'param': {'size': [5, 1], 'edge_func': 0}, 'wpsnr': 42.223749330901626, 'detection': 0}
{'image': '0087.bmp', 'attack': 'median_edge', 'param': {'size': [7, 1], 'edge_func': 0}, 'wpsnr': 37.858252934914546, 'detection': 0}
{'image': '0087.bmp', 'attack': 'median_edge', 'param': {'size': [9, 1], 'edge_func': 0}, 'wpsnr': 35.604925367351065, 'detection': 0}
{'image': '0087.bmp', 'attack': 'median_edge', 'param': {'size': [3, 9], 'edge_func': 1}, 'wpsnr': 44.13541214701581, 'detection': 0}
{'image': '0087.bmp', 'attack': 'medi

3. Blur flat regions using gaussian filters

In [40]:
progress_bar = tqdm(watermarked, desc = "Blurring edges of " + str(N) + " images")
history = []
successful_attacks = []
total = 0
succ = 0
avg = 0
max = 0

for index, wm in enumerate(progress_bar):
    for edge_func in range(0, 2):
        for sigma in np.arange(0, 5, 0.1, dtype = np.float16):        
            attacked, attack_name, usd = grt.gauss_flat(wm, sigma, edge_func)
            detected = detection.detection(images[index], wm, attacked, alpha, max_layer)
            wpsnr_attacked = utility.wpsnr(wm, attacked)
            log = {"image" : img_files[index],
                    "attack" : attack_name,
                    "param" : usd,
                    "wpsnr" : wpsnr_attacked,
                    "detection" : detected}
            history.append(log)
            if detected == 0 and wpsnr >= 35:
                total += wpsnr_attacked
                succ +=1
                successful_attacks.append(log)
                if wpsnr_attacked > max:
                    max = wpsnr_attacked
                break

if succ > 0:
    avg = total/succ
    print("\nAverage wpsnr of successful attacks: " + str(avg) + ' --> ' + str(robustness(avg)) + ' robustness points')
    print("Max wpsnr of successful attacks: " + str(max) + ' --> ' + str(robustness(max)) + ' robustness points\n') 
else:
    print("\nThis attack is not very effective...\n")
 
for s in successful_attacks:
    print(s) 

Blurring edges of 2 images: 100%|██████████| 2/2 [03:24<00:00, 102.42s/it]


Average wpsnr of successful attacks: 52.125245009188376 --> 1 robustness points
Max wpsnr of successful attacks: 56.368261742338575 --> 0 robustness points

{'image': '0087.bmp', 'attack': 'gauss_flat', 'param': {'sigma': 0.5, 'edge_func': 1}, 'wpsnr': 56.368261742338575, 'detection': 0}
{'image': '0029.bmp', 'attack': 'gauss_flat', 'param': {'sigma': 0.5996, 'edge_func': 1}, 'wpsnr': 47.88222827603818, 'detection': 0}


4. Blur flat regions using median filters

In [41]:
progress_bar = tqdm(watermarked, desc = "Blurring edges of " + str(N) + " images")
history = []
successful_attacks = []
total = 0
succ = 0
avg = 0
max = 0

for index, wm in enumerate(progress_bar):
    for edge_func in range(0, 2):
        for height in np.arange(1, 15, 2):
            for width in np.arange(1, 15, 2):       
                attacked, attack_name, usd = grt.median_flat(wm, [height, width], edge_func)
                detected = detection.detection(images[index], wm, attacked, alpha, max_layer)
                wpsnr_attacked = utility.wpsnr(wm, attacked)
                log = {"image" : img_files[index],
                        "attack" : attack_name,
                        "param" : usd,
                        "wpsnr" : wpsnr_attacked,
                        "detection" : detected}
                history.append(log)
                if detected == 0 and wpsnr >= 35:
                    total += wpsnr_attacked
                    succ +=1
                    successful_attacks.append(log)
                    if wpsnr_attacked > max:
                        max = wpsnr_attacked
                    break

if succ > 0:
    avg = total/succ
    print("\nAverage wpsnr of successful attacks: " + str(avg) + ' --> ' + str(robustness(avg)) + ' robustness points')
    print("Max wpsnr of successful attacks: " + str(max) + ' --> ' + str(robustness(max)) + ' robustness points\n') 
else:
    print("\nThis attack is not very effective...\n")
 
for s in successful_attacks:
    print(s) 

Blurring edges of 2 images: 100%|██████████| 2/2 [08:31<00:00, 255.60s/it]


Average wpsnr of successful attacks: 40.12834733278909 --> 5 robustness points
Max wpsnr of successful attacks: 48.62189236596774 --> 2 robustness points

{'image': '0087.bmp', 'attack': 'median_flat', 'param': {'size': [1, 7], 'edge_func': 1}, 'wpsnr': 42.97391566249702, 'detection': 0}
{'image': '0087.bmp', 'attack': 'median_flat', 'param': {'size': [3, 3], 'edge_func': 1}, 'wpsnr': 48.62189236596774, 'detection': 0}
{'image': '0087.bmp', 'attack': 'median_flat', 'param': {'size': [5, 3], 'edge_func': 1}, 'wpsnr': 41.79203923749326, 'detection': 0}
{'image': '0087.bmp', 'attack': 'median_flat', 'param': {'size': [7, 1], 'edge_func': 1}, 'wpsnr': 38.04308875788846, 'detection': 0}
{'image': '0087.bmp', 'attack': 'median_flat', 'param': {'size': [9, 1], 'edge_func': 1}, 'wpsnr': 35.9135009030011, 'detection': 0}
{'image': '0029.bmp', 'attack': 'median_flat', 'param': {'size': [3, 5], 'edge_func': 1}, 'wpsnr': 36.82974742009318, 'detection': 0}
{'image': '0029.bmp', 'attack': 'median_f

5. Adding white gaussian noise (AWGN) to edges

In [42]:
progress_bar = tqdm(watermarked, desc = "Applying AWGN to edges of " + str(N) + " images")
history = []
successful_attacks = []
total = 0
succ = 0
avg = 0
max = 0

for index, wm in enumerate(progress_bar):
    for edge_func in range(0, 2):
        for std in range(0, 110, 10):      
            attacked, attack_name, usd = grt.awgn_edge(wm, 0, std, 100, edge_func)
            detected = detection.detection(images[index], wm, attacked, alpha, max_layer)
            wpsnr_attacked = utility.wpsnr(wm, attacked)
            log = {"image" : img_files[index],
                    "attack" : attack_name,
                    "param" : usd,
                    "wpsnr" : wpsnr_attacked,
                    "detection" : detected}
            history.append(log)
            if detected == 0 and wpsnr >= 35:
                total += wpsnr_attacked
                succ +=1
                successful_attacks.append(log)
                if wpsnr_attacked > max:
                    max = wpsnr_attacked
                break

if succ > 0:
    avg = total/succ
    print("\nAverage wpsnr of successful attacks: " + str(avg) + ' --> ' + str(robustness(avg)) + ' robustness points')
    print("Max wpsnr of successful attacks: " + str(max) + ' --> ' + str(robustness(max)) + ' robustness points\n') 
else:
    print("\nThis attack is not very effective...\n")
 
for s in successful_attacks:
    print(s) 

Applying AWGN to edges of 2 images: 100%|██████████| 2/2 [00:48<00:00, 24.13s/it]


Average wpsnr of successful attacks: 41.00547813421608 --> 4 robustness points
Max wpsnr of successful attacks: 45.35693156070433 --> 3 robustness points

{'image': '0087.bmp', 'attack': 'awgn_edge', 'param': {'mean': 0, 'std': 20, 'seed': 100, 'edge_func': 0}, 'wpsnr': 45.35693156070433, 'detection': 0}
{'image': '0087.bmp', 'attack': 'awgn_edge', 'param': {'mean': 0, 'std': 30, 'seed': 100, 'edge_func': 1}, 'wpsnr': 39.464012971328025, 'detection': 0}
{'image': '0029.bmp', 'attack': 'awgn_edge', 'param': {'mean': 0, 'std': 40, 'seed': 100, 'edge_func': 0}, 'wpsnr': 38.19548987061588, 'detection': 0}


### Combo attacks

1. Resize a JPEG-compressed watermarked image

In [94]:
attacked, attack_name, usd = grt.resize_jpeg(watermarked[1], 45, 0.999999999)
detected = detection.detection(images[1], watermarked[1], attacked, alpha, max_layer)
wpsnr_attacked = utility.wpsnr(watermarked[1], attacked)
log = {"image" : img_files[1],
                    "attack" : attack_name,
                    "param" : usd,
                    "wpsnr" : wpsnr_attacked,
                    "detection" : detected}

print(log)

{'image': '0029.bmp', 'attack': 'resize_jpeg', 'param': {'qf': 45, 'scale': 0.999999999}, 'wpsnr': 50.6471425829405, 'detection': 0}


2. Blur a JPEG-compressed watermarked image using gaussian filters

In [ ]:
# Coming soon...

3. Blur a JPEG-compressed watermarked image using median filters

In [ ]:
# Coming soon...

4. Blur after AWGN using gaussian filters

In [ ]:
# Coming soon...

5. Blur after AWGN using median filters

In [ ]:
# Coming soon...

6. JPEG compression after AWGN

In [ ]:
# Coming soon...